<a href="https://colab.research.google.com/github/mkbond777/DTI-meta-learning/blob/master/Modular_code_custom_cnn_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-addons

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

import json

import re

import zipfile
import os

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import confusion_matrix, f1_score

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import tensorflow as tf

import matplotlib.pyplot as plt

import seaborn as sns

from shutil import copyfile

import datetime

import tensorflow_addons as tfa

from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import time

import glob

import random


In [ ]:
#chembl_id = 'CHEMBL286'

In [ ]:
#! rm -r /content/model_init_CHEMBL286

In [ ]:
def main(src_df, target_ids,metrics):

  curr_dt_time = datetime.datetime.now()

  # unzipping entire image data 
  src_img_path = '/content/drive/MyDrive/ML-DTI/target_training_datasets.zip'
  dest_img_path = '/content/img_path'
  if not os.path.exists(dest_img_path):
    os.mkdir(dest_img_path) 
    unzip_img_data(src_img_path,dest_img_path)

  out_file = '/content/drive/MyDrive/ML-DTI/metrics_' + 'custom' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '.csv'

  for target in target_ids:
    target_df = target_specific_data(src_df,target)
    target_df.reset_index(drop=True,inplace=True)
    target_df['labels'] = target_df['labels'].astype(str)
    target_specific_run(target_df, target, out_file, metrics)



In [ ]:
def target_specific_run(target_df, target_id, filename,metrics):

  # imagedatagenerator for CNN
  train_generator, valid_generator, test_generator = prepare_train_valid_test_cnn_data(target_df, target_id)

  cnn_model = custom_cnn_model()

  cnn_model = compile_cnn_model(cnn_model, metrics)

  cl = callback_list(cnn_model, metrics, target_id)

  cnn_model, history = cnn_model_training(50, cnn_model, train_generator, valid_generator, cl)

  model_weights = get_latest_file(target_id)


  cnn_model = load_weights(cnn_model, model_weights)

  train_metrics = log_metrics(cnn_model, train_generator)

  test_metrics = log_metrics(cnn_model, test_generator)

  cnn_predict_df = predictions(cnn_model, target_df, target_id)

  encoded_df = label_encoding_smiles(target_df)

  svm_cnn_acc,svm_cnn_auc,svm_cnn_f1_score = svm_model_training(encoded_df, cnn_predict_df, metrics, with_cnn = True)

  l = [target_id,train_metrics[1],train_metrics[2],train_metrics[3][0],test_metrics[1],test_metrics[2],test_metrics[3][0],svm_cnn_acc,svm_cnn_auc,svm_cnn_f1_score]

  data = ",".join([str(i) for i in l])
  
  with open(filename, "a") as myfile:
    myfile.write(data + "\n")

  delete_folder(target_id)


In [ ]:
def delete_folder(target_id):
  folder_name = 'model_init' + '_' + target_id + '/'
  !rm -r {folder_name}

In [ ]:
def svm_model_training(encoded_df, cnn_predict_df, metrics, with_cnn = False):
  svm_train_df = prepare_svm_data(encoded_df[encoded_df['type'] != 'test'], cnn_predict_df, with_cnn)

  svm_test_df = prepare_svm_data(encoded_df[encoded_df['type'] == 'test'], cnn_predict_df, with_cnn)

  model_svm = train_svm(svm_train_df)

  return test_svm(model_svm, svm_test_df, metrics)

In [ ]:
def get_latest_file(target_id):
  folder_path = 'model_init' + '_' + target_id + '/*'
  list_of_files = glob.glob(folder_path) # * means all if need specific format then *.csv
  latest_file = max(list_of_files, key=os.path.getctime)
  return latest_file

In [ ]:
def unzip_img_data(src_path,dest_path):
  if dest_path is None:
    !unzip -qq {src_path}
  else:
    !unzip -qq {src_path} -d {dest_path}

In [ ]:
def prepare_data():

  # create a column for img path
  def f(row):
    return row['drug_id'] + '.png'

  # read json files containing test, train and valid
  json_df = pd.read_csv('/content/drive/MyDrive/ML-DTI/json_df.csv')
  
  # read smiles files
  smiles_df = pd.read_csv('/content/drive/MyDrive/ML-DTI/chembl_v28_json_joined_202110241711.csv')

  final_df = pd.merge(json_df, smiles_df, left_on=['drug_id',], right_on=['drug_id'],how='inner')

  final_df['img_path'] = final_df.apply(f, axis=1)

  return final_df

  


In [ ]:
def target_specific_data(df, target_id):
  # filter data based on given chembl_id
  return df[df['target_id'] == target_id]

In [ ]:
def prepare_train_valid_test_cnn_data(df, target_id, img_h = 200, img_w = 200):
  
  print(target_id + '\n')
  
  zip_path = '/content/img_path/' + target_id + '.zip'
  img_path = '/content/img_path/' + target_id + '/imgs/'


  if not os.path.exists(img_path): 
    unzip_img_data(zip_path,'/content/img_path/')

  datagen=ImageDataGenerator(validation_split=0.2)

  train_generator=datagen.flow_from_dataframe(
    dataframe=df[df['type'] != 'test'],
    directory=img_path,
    x_col="img_path",
    y_col="labels",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="binary",
    target_size=(img_h,img_w))
  
  valid_generator=datagen.flow_from_dataframe(
    dataframe=df[df['type'] != 'test'],
    directory=img_path,
    x_col="img_path",
    y_col="labels",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="binary",
    target_size=(img_h,img_w))
  
  test_datagen=ImageDataGenerator()
  
  test_generator=test_datagen.flow_from_dataframe(
    dataframe=df[df['type'] == 'test'],
    directory=img_path,
    x_col="img_path",
    y_col="labels",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="binary",
    target_size=(img_h,img_w))
  
  return train_generator, valid_generator, test_generator

In [ ]:
# def pre_trained_cnn_model(name, img_height=200,img_width=200):
#   # Configure the dataset for performance
#   #AUTOTUNE = tf.data.AUTOTUNE

#   #train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
#   # train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#   # val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

#   preprocess_input, base_model = pre_trained_cnn_model_selection(name)
  
#   global_average_layer = tf.keras.layers.GlobalMaxPooling2D()

#   prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

#   base_model.trainable = False

#   inputs = tf.keras.Input(shape=(img_height, img_width, 3))
#   x = preprocess_input(inputs)
#   x = base_model(x, training=False)
#   x = global_average_layer(x)
#   x = tf.keras.layers.Dropout(0.2)(x)

#   outputs = prediction_layer(x)
#   model = tf.keras.Model(inputs, outputs)

#   return model


In [ ]:
def custom_cnn_model(img_height=200,img_width=200):
    model = Sequential([
      layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(1, activation='sigmoid')
    ])

    return model



In [ ]:
# def pre_trained_cnn_model_selection(name, img_height=200,img_width=200):

#   if 'vgg16' in name.lower():
#     preprocess_input = tf.keras.applications.vgg16.preprocess_input

#     base_model = tf.keras.applications.vgg16.VGG16(input_shape=(img_height, img_width, 3),
#                                                   include_top=False,
#                                                   weights='imagenet')
#     return preprocess_input, base_model
    
#   if 'inception' in name.lower():
#     preprocess_input = tf.keras.applications.inception_v3.preprocess_input

#     base_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(img_height, img_width, 3),
#                                                   include_top=False,
#                                                   weights='imagenet')
#     return preprocess_input, base_model


    
#   preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

#   base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(img_height, img_width, 3),
#                                               include_top=False,
#                                               weights='imagenet')
  
#   return preprocess_input, base_model
    
  
  

In [ ]:
def compile_cnn_model(model, metric_list, base_rate = 0.0001):
  metrics = get_metrics(metric_list)

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=metrics)
  return model

In [ ]:
def get_metrics(metric_list):
  metric_list_lower = [i.lower() for i in metric_list]
  metrics = []
  if 'accuracy' in metric_list_lower :
    metrics.append('accuracy')
  if 'auc' in metric_list_lower :
    metrics.append(tf.keras.metrics.AUC(name='auc'))
  if 'f1_score' in metric_list_lower:
    metrics.append(tfa.metrics.F1Score(num_classes=1,threshold=0.5,
                                       name='f1_score'))
  
  return metrics


In [ ]:
def callback_list(model, metric_list, target_id):

  metric_list_1st = metric_list[0].lower()

  model_name = 'model_init' + '_' + target_id + '/'

  if not os.path.exists(model_name):
    os.mkdir(model_name)

  filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{auc:.5f}-{val_loss:.5f}-{val_auc:.5f}.h5'

  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                  monitor='val_auc', 
                                                  verbose=1, 
                                                  save_best_only=True, 
                                                  save_weights_only=True, 
                                                  mode='max', 
                                                  save_freq='epoch')

  es = tf.keras.callbacks.EarlyStopping(monitor='val_auc',
                                        mode='max', 
                                        verbose=1, 
                                        patience=5)
  #callbacks_list = [checkpoint, LR, es]
  callbacks_list = [checkpoint, es]

  return callbacks_list


In [ ]:
def cnn_model_training(epoch, model, train_ds, val_ds, callbacks_list):

  history = model.fit(train_ds,
                      epochs=epoch,
                      validation_data=val_ds,
                      callbacks=callbacks_list)
  return model, history

In [ ]:
def load_weights(model, model_weights):
  model.load_weights(model_weights)
  return model

In [ ]:
def log_metrics(model, ds):
  return model.evaluate(ds, verbose=1)

In [ ]:
def predictions(model, df, target_id, img_height=200,img_width=200):
  data_list = []
  imgs_path = '/content/img_path/' + target_id + '/imgs/'

  for index, row in df.iterrows():
    img_path = os.path.join(imgs_path, row['img_path'])
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    #img_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(img_batch)
    prediction = model.predict(img_batch)
    data_list.append((row['drug_id'],row['labels'],prediction[0][0]))

  df = pd.DataFrame(data_list,columns=['drug_id','label','y_pred_prob',])  

  return df

In [ ]:
def label_encoding_smiles(df):

  CHARCANSMISET = { "#": 1, "%": 2, ")": 3, "(": 4, "+": 5, "-": 6, 
         ".": 7, "1": 8, "0": 9, "3": 10, "2": 11, "5": 12, 
         "4": 13, "7": 14, "6": 15, "9": 16, "8": 17, "=": 18, 
         "A": 19, "C": 20, "B": 21, "E": 22, "D": 23, "G": 24,
         "F": 25, "I": 26, "H": 27, "K": 28, "M": 29, "L": 30, 
         "O": 31, "N": 32, "P": 33, "S": 34, "R": 35, "U": 36, 
         "T": 37, "W": 38, "V": 39, "Y": 40, "[": 41, "Z": 42, 
         "]": 43, "_": 44, "a": 45, "c": 46, "b": 47, "e": 48, 
         "d": 49, "g": 50, "f": 51, "i": 52, "h": 53, "m": 54, 
         "l": 55, "o": 56, "n": 57, "s": 58, "r": 59, "u": 60,
         "t": 61, "y": 62, "/" : 63, "\\" : 64, "@":65}

  CHARCANSMILEN = 65

  def one_hot_sequence(line, MAX_SEQ_LEN = 80, smi_ch_ind = CHARCANSMISET):
    X = np.zeros((MAX_SEQ_LEN, len(smi_ch_ind))) 
    for i, ch in enumerate(line[:MAX_SEQ_LEN]):
        X[i,(smi_ch_ind[ch])-1] = 1

    return X.flatten() #.tolist()

  df['lbl_encoding'] = df.apply(
      lambda row : one_hot_sequence(row['canonical_smiles']), axis = 1)

  return df


In [ ]:
def prepare_svm_data(df, cnn_predict_df,meta_learning=False):

  df.reset_index(drop=True,inplace=True)

  def f(row):
    if row['y_pred_prob'] <= 0.5 :
        val = 0
    else:
        val = 1
    return val

  def svm_cnn_date(df, cnn_predict_df):
    cnn_predict_df['y_pred'] = cnn_predict_df.apply(f,axis=1)
    return pd.merge(df, cnn_predict_df, left_on=['drug_id',], 
                          right_on=['drug_id'],how='inner')
    
  if meta_learning:
    final_svm_df = svm_cnn_date(df, cnn_predict_df)
    final_svm_df = pd.concat([pd.DataFrame(final_svm_df.lbl_encoding.values.tolist()), final_svm_df.y_pred, final_svm_df.labels],axis=1)
  else:
    final_svm_df = pd.concat([pd.DataFrame(df.lbl_encoding.values.tolist()), df.labels],axis=1)
  
  final_svm_df = final_svm_df.astype('uint8')

  return final_svm_df


In [ ]:
def train_svm(svm_df, kernel_type = 'rbf'):
  X_train = svm_df.drop("labels", axis = 1)
  y_train = svm_df['labels']

  model = SVC(kernel=kernel_type, probability=True)
  model.fit(X_train, y_train)

  return model

In [ ]:
def test_svm(model, svm_df_test, metrics):
  X_test = svm_df_test.drop("labels", axis = 1)
  y_test = svm_df_test['labels']

  y_pred = model.predict(X_test)

  accuracy = round(accuracy_score(y_test,y_pred),2)
  roc_auc = round(roc_auc_score(y_test,y_pred),2) 
  pred_f1_score = round(f1_score(y_test,y_pred),2)
  return accuracy, roc_auc, pred_f1_score

### Preparing data and running model

In [ ]:
src_df = prepare_data()

In [ ]:
l = ['CHEMBL223','CHEMBL3473','CHEMBL228','CHEMBL276','CHEMBL3568','CHEMBL1900','CHEMBL4822','CHEMBL1981','CHEMBL2069','CHEMBL3024','CHEMBL3231','CHEMBL2959','CHEMBL2742','CHEMBL1908389','CHEMBL4578','CHEMBL1785','CHEMBL1994','CHEMBL3286','CHEMBL4128','CHEMBL206','CHEMBL4308','CHEMBL257','CHEMBL2993','CHEMBL2039','CHEMBL2292','CHEMBL208','CHEMBL2581','CHEMBL1855','CHEMBL2028','CHEMBL6136','CHEMBL2413','CHEMBL3571','CHEMBL2722','CHEMBL2695','CHEMBL298','CHEMBL1821','CHEMBL213','CHEMBL2014','CHEMBL304','CHEMBL2001','CHEMBL3522','CHEMBL2949','CHEMBL1946','CHEMBL5147','CHEMBL3974','CHEMBL3920','CHEMBL1867','CHEMBL288','CHEMBL2016','CHEMBL3973','CHEMBL2598','CHEMBL3358','CHEMBL1835','CHEMBL1978','CHEMBL4588','CHEMBL216','CHEMBL2431','CHEMBL281','CHEMBL3553','CHEMBL1875','CHEMBL4204','CHEMBL2808','CHEMBL229','CHEMBL1936','CHEMBL331','CHEMBL3594','CHEMBL4794','CHEMBL2820','CHEMBL338','CHEMBL3045','CHEMBL2525','CHEMBL6164','CHEMBL3142','CHEMBL3649','CHEMBL5407','CHEMBL2035','CHEMBL1991','CHEMBL4908','CHEMBL2208','CHEMBL221','CHEMBL321','CHEMBL4422','CHEMBL3979','CHEMBL265','CHEMBL3976','CHEMBL3869','CHEMBL2047','CHEMBL335','CHEMBL2276','CHEMBL324','CHEMBL1801','CHEMBL231','CHEMBL308','CHEMBL3629','CHEMBL313']

In [ ]:
len(target_ids)

In [ ]:
metrics = ['accuracy','auc','f1_score']

In [ ]:
start_time = time.time()
main(src_df, target_ids,metrics)
print("--- %s seconds ---" % (time.time() - start_time))